<img src="../../images/BikeDNA_logo.svg" width="250"  alt="BikeDNA logo" style="display:block; margin-left: auto; margin-right: auto;">
<a href="https://github.com/anerv/BikeDNA">Github</a>

# Example reference data preprocessing: GeoDanmark

This notebook provides an example of how a spatial dataset with data on cycling infrastructure can be converted to the format required by BikeDNA. When using your own data, The preprocessing must be adapted to content and format.

The data used in this notebook are from *GeoDanmark* and were downloaded from [dataforsyningen.dk](https://dataforsyningen.dk/) under the [GeoDanmark license](https://www.geodanmark.dk/wp-content/uploads/2022/08/Vilkaar-for-brug-af-frie-geografiske-data_GeoDanmark-grunddata-august-2022.pdf).

As stated in the data set requirements, the reference data should:

- only contain **cycling infrastructure** (i.e. not also the regular street network)
- have all geometries as **LineStrings** (not MultiLineString)
- for each row, the geometry should be a **straight** LineString only defined by its start- and end nodes
- have start/end nodes at **intersections**
- be in a **CRS** recognised by GeoPandas
- contain a column describing whether each feature is a physically **protected**/separated infrastructure or if it is **unprotected**
- contain a column describing whether each feture is **bidirectional** or not
- contain a column describing how features have been **digitized** ('geometry type')
- contain a column with a unique **ID** for each feature

In [1]:
import contextily as cx
import folium
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import matplotlib.pyplot as plt
import momepy
from shapely.ops import linemerge

from src import graph_functions as gf
from src import plotting_functions as pf

%run ../settings/tiledict.py


/opt/miniconda3/envs/bikedna_denmark/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
geodk = gpd.read_parquet("../../data/REFERENCE/dk/raw/geodk_bike.parquet")
geodk.sample(10)

,gml_id,objectid,id_lokalid,id_namespace,tempid,geometristatus,virkningfra,virkningtil,virkningsaktoer,registreringfra,...,kommunekode,vejkode,vejkategori,trafikart,niveau,overflade,tilogfrakoersel,rundkoersel,gmlid,geometry
187717,id2eb8c8f9-fadf-4f7e-be56-69f3185d1064,108734517320151125151450,1087345173,http://data.gov.dk/geodanmark,None,Endelig,20151125151450,None,KMS,20151125151450,...,0101,5988,Cykelsti langs vej,Cykelsti,None,Befæstet,False,False,gdk60.1087345173,MULTILINESTRING Z ((723345.760 6179713.190 8.9...
2150349,idb250160c-f174-4310-8109-56ead3e86450,108736828720220513205218715,1087368287,http://data.gov.dk/geodanmark,None,Endelig,20190409172557,None,COWI,20220513205218.715,...,0101,3332,Cykelsti langs vej,Cykelsti,None,Befæstet,False,False,gdk60.1087368287,MULTILINESTRING Z ((724638.990 6179454.420 9.8...
1377868,idbb8161be-3c5c-4242-85bb-9d35511126c1,110924222720170817090011,1109242227,http://data.gov.dk/geodanmark,None,Endelig,20170817090011,None,COWI,20170817090011,...,0330,0123,Cykelbane langs vej,Cykelsti,None,Befæstet,False,False,gdk60.1109242227,MULTILINESTRING Z ((641394.550 6143698.990 8.7...
1017416,id79ef2c7e-b9b1-4a7b-9c1d-0c52000c0093,111115917020171016090432,1111159170,http://data.gov.dk/geodanmark,None,Endelig,20171016090432,None,COWI,20171016090432,...,0265,4816,Cykelbane langs vej,Cykelsti,None,Befæstet,False,False,gdk60.1111159170,MULTILINESTRING Z ((690805.660 6168958.840 44....
1390652,id20590f0f-ceff-43c1-94b0-f7591eb62365,108315837920151103102119,1083158379,http://data.gov.dk/geodanmark,None,Endelig,20151103102119,None,GST,20151103102119,...,0791,1208,Cykelsti langs vej,Cykelsti,None,Befæstet,False,False,gdk60.1083158379,MULTILINESTRING Z ((546799.330 6262042.980 53....
183504,id04850ef7-0540-49c8-94e6-0e9fe7bcac34,111053975520170920133641,1110539755,http://data.gov.dk/geodanmark,None,Endelig,20170920133641,None,COWI,20170920133641,...,0253,3036,Cykelbane langs vej,Cykelsti,None,Befæstet,False,False,gdk60.1110539755,MULTILINESTRING Z ((703089.980 6163049.020 19....
821277,iddc9a640a-3669-422a-84b0-f59661134a92,110777382120170425142642,1107773821,http://data.gov.dk/geodanmark,None,Endelig,20170425142642,None,COWI,20170425142642,...,0751,None,Cykelsti langs vej,Cykelsti,None,Befæstet,False,False,gdk60.1107773821,MULTILINESTRING Z ((573372.720 6227836.000 69....
1561582,id3ca3ab41-08be-47a6-9378-00c5f2fe43aa,110135283220161018125513,1101352832,http://data.gov.dk/geodanmark,None,Endelig,20161018125513,None,Cowi,20161018125513,...,0787,9029,Cykelsti langs vej,Cykelsti,None,Befæstet,False,False,gdk60.1101352832,MULTILINESTRING Z ((461514.080 6312587.510 7.6...
1818093,id8598fdb9-a7c6-4e6c-adbc-f9becc2555e3,110514861620170130143012,1105148616,http://data.gov.dk/geodanmark,None,Endelig,20170130143012,None,Cowi,20170130143012,...,0851,6985,Cykelsti langs vej,Cykelsti,None,Befæstet,False,False,gdk60.1105148616,MULTILINESTRING Z ((558628.860 6324145.240 3.9...
156264,id8e3d1e14-41c7-4f51-98d9-cfc252f7ebe5,107643732420150715005401,1076437324,http://data.gov.dk/geodanmark,None,Endelig,20150715005401,None,GST,20150715005401,...,0157,0802,Cykelbane langs vej,Cykelsti,None,Befæstet,False,False,gdk60.1076437324,MULTILINESTRING Z ((724527.450 6182889.120 4.0...


Our dataset contains the entire road network, including bicycle tracks and lanes. We are only interested in the dedicated cycling infrastructure and thus need to select a subset of the data.
We also only want to include infrastructure that is completed or under construction.

Some of the data might be outside of the study area we are interested in, but the data processing in notebook 01 will clip all data to the desired extent.

In [3]:
# Creating subset only with existing cycling infrastructure

geodk_selection = geodk.loc[
    (geodk.vejkategori.isin(["Cykelsti langs vej", "Cykelbane langs vej"]))
    & (geodk.status.isin(["Anlagt", "Under anlæg"]))
].copy()

In [4]:
geodk_selection.explore()

For all code to run without errors, our dataset can only contain LineString geometries. Let's check what we have:

In [5]:
geodk_selection.geom_type.unique()

array(['MultiLineString'], dtype=object)

In this dataset, we only have MultiLineStrings. To fix this, we first try to merge the MultiLineStrings. 
If some of the MultiLinestrings are not connected (i.e. there are gaps in the lines), the aboves step will not be able to merge them. In that case we can instead 'explode' them.

In [6]:
geodk_linestrings = geodk_selection.copy()
# Convert MultiLineStrings to LineString
geodk_linestrings["geometry"] = geodk_linestrings["geometry"].apply(
    lambda x: linemerge(x) if x.geom_type == "MultiLineString" else x
)

if (
    len(geodk_linestrings.geom_type.unique()) > 1
    or geodk_linestrings.geom_type.unique()[0] != "LineString"
):

    print("Exploding MultiLineStrings...")
    geodk_linestrings = geodk_selection.explode(ignore_index=True)

assert len(geodk_linestrings.geom_type.unique()) == 1
assert geodk_linestrings.geom_type.unique()[0] == "LineString"
geodk_linestrings.geom_type.unique()

array(['LineString'], dtype=object)

For the code to work, the data need to be in a CRS recognized by GeoPandas, and to have that CRS defined. Let's check that we have a CRS defined:

In [7]:
geodk_linestrings.crs

<Derived Projected CRS: EPSG:25832>
Name: ETRS89 / UTM zone 32N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe between 6°E and 12°E: Austria; Belgium; Denmark - onshore and offshore; Germany - onshore and offshore; Norway including - onshore and offshore; Spain - offshore.
- bounds: (6.0, 38.76, 12.01, 84.33)
Coordinate Operation:
- name: UTM zone 32N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

The analysis of data quality is based on the concept of a *network*. For the results to be accurate we need a dataset with nodes at intersections (i.e. where the lines defining the cycling infrastructure intersect).

Use the folium plot below to check that you do have nodes at intersections.
If not, this will have to be fixed - or it will be an aspect of low data quality that will become apparent in the analysis of data quality...

Don't worry if there are more nodes than just those at intersections and start/end points - we will take care of that in the data loading notebook.

In [8]:
G = momepy.gdf_to_nx(
    geodk_linestrings.to_crs("EPSG:25832"), approach="primal", directed=True
)  # We reproject the network data to avoid warnings - final reprojection will happen later

nodes, edges = momepy.nx_to_gdf(G)

In [9]:
# Feature groups for OSM
edges_folium = pf.make_edgefeaturegroup(
    gdf=edges, mycolor="black", myweight=2, nametag="edges", show_edges=True
)

nodes_folim = pf.make_nodefeaturegroup(
    gdf=nodes, mycolor="red", mysize=2, nametag="nodes", show_nodes=True
)

feature_groups = [edges_folium, nodes_folim]

m = pf.make_foliumplot(
    feature_groups=feature_groups,
    layers_dict=folium_layers,
    center_gdf=nodes,
    center_crs=nodes.crs,
)

display(m)

We don't technically need to drop any unnecessary columns, but let's avoid loading unnecessary data later on.

In [10]:
geodk_linestrings.columns

Index(['gml_id', 'objectid', 'id_lokalid', 'id_namespace', 'tempid',
       'geometristatus', 'virkningfra', 'virkningtil', 'virkningsaktoer',
       'registreringfra', 'registreringtil', 'registreringsaktoer',
       'forretningsomraade', 'forretningshaendelse', 'forretningsproces',
       'status', 'registreringsspecifikation', 'dataansvar',
       'plannoejagtighed', 'vertikalnoejagtighed', 'planstedfaestelsesmetode',
       'vertikalstedfaestelsesmetode', 'kommentar', 'applikation',
       'vejmidtetype', 'vejmyndighed', 'cvfadmnr', 'kommunekode', 'vejkode',
       'vejkategori', 'trafikart', 'niveau', 'overflade', 'tilogfrakoersel',
       'rundkoersel', 'gmlid', 'geometry'],
      dtype='object')

In [11]:
# Drop unnecessary columns

geodk_linestrings = geodk_linestrings[["id_lokalid", "vejkategori", "geometry"]]

For consistency, we rename all column names to lower case letters:

In [12]:
geodk_linestrings = gf.clean_col_names(geodk_linestrings)

For this dataset we assume of all features to be 'true' geometry mappings and one directional, so we can specify this in config file and do not have to add it to the data.

The rest of the pre-processing, such as projecting to the chosen CRS, clipping the data to the study area etc. will happen in [notebook 2a](../REFERENCE/2a_initialize_reference.ipynb).

**Final dataset**

In [13]:
geodk_linestrings.sample(10)

,id_lokalid,vejkategori,geometry
2279846,1112131687,Cykelsti langs vej,"LINESTRING Z (670470.160 6176865.350 26.640, 6..."
1862843,1067416190,Cykelbane langs vej,"LINESTRING Z (599386.950 6108443.050 67.150, 5..."
596539,1100615250,Cykelbane langs vej,"LINESTRING Z (517731.680 6255916.420 18.190, 5..."
766714,1222561493,Cykelbane langs vej,"LINESTRING Z (508207.480 6199470.210 47.970, 5..."
1489694,1081218937,Cykelbane langs vej,"LINESTRING Z (709418.070 6183811.080 39.470, 7..."
35472,1095950079,Cykelsti langs vej,"LINESTRING Z (508780.960 6158284.990 65.440, 5..."
156175,1076437088,Cykelsti langs vej,"LINESTRING Z (724764.500 6186822.860 3.650, 72..."
2285021,1215814889,Cykelbane langs vej,"LINESTRING Z (585114.640 6139751.580 16.880, 5..."
931188,1079714199,Cykelsti langs vej,"LINESTRING Z (719516.620 6186136.890 22.910, 7..."
1962024,1213798546,Cykelbane langs vej,"LINESTRING Z (541898.550 6090856.050 34.600, 5..."


**Export dataset**

In [14]:
geodk_linestrings.to_parquet(
    "../../data/reference/dk/processed/reference_data.parquet"
)



*Contains data from GeoDanmark (retrieved spring 2023)*
*© SDFE (Styrelsen for Dataforsyning og Effektivisering og Danske kommuner)*

*License: [GeoDanmark](https://www.geodanmark.dk/wp-content/uploads/2022/08/Vilkaar-for-brug-af-frie-geografiske-data_GeoDanmark-grunddata-august-2022.pdf)*